>For Googla Colab Only

>>git clone https://github.com/OperationalizingAI/Hackathon-2-22-24.gi`


In [ ]:
!pip install -r requirements.txt

### Google Only Code

In [ ]:
!pip install google-cloud-secret-manager
!pip install --upgrade google-auth

import os

from google.cloud import secretmanager
from google.colab import auth
from google.colab import drive

In [ ]:
def load_secrets(secrets_name, project_id):
  # Build a client
  auth.authenticate_user()
  client = secretmanager.SecretManagerServiceClient()
  secret_name = secrets_name
  # Create path to latest secret
  resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/latest"
  # Get your secret :
  response = client.access_secret_version(request={"name": resource_name})
  secret_string = response.payload.data.decode('UTF-8')
  return secret_string

In [ ]:
project_id = 'botchagalupep1'
openai_api_key = load_secrets("openai_api_key",project_id)
os.environ['OPENAI_API_KEY'] = openai_api_key
#MONGODB_ATLAS_CLUSTER_URI = load_secrets("mdb_uri",project_id)
MONGODB_ATLAS_CLUSTER_URI = load_secrets("MDB_CLUSTER0_URI",project_id)
langsmith_api_key = load_secrets("langsmith_api_key",project_id)
#print(langsmith_api_key )
#print(MONGODB_ATLAS_CLUSTER_URI)

In [ ]:
DB_NAME = 'langchain_db'
COLLECTION_NAME = 'test'
INDEX_NAME = 'vector_index'

In [ ]:
from AtlasClient import AtlasClient

atlas_client = AtlasClient (MONGODB_ATLAS_CLUSTER_URI, DB_NAME)
print("Connected to the Mongo Atlas database!")

Connected to the Mongo Atlas database!


In [ ]:
from OpenAIClient import OpenAIClient

openAI_client = None

openAI_client = OpenAIClient (api_key=openai_api_key)
print ("OpenAI client initialized")

OpenAI client initialized


In [ ]:
queries = [

'What is GPT-4?',
'What is GPT-3.5?',
'What is GPT-3.5 in terms of capabilities?',
'What is GPT-4 in terms of capabilities?'

]


In [ ]:
embeddings = {}

for query in queries:
    embedding  = openAI_client.get_embedding(query, model='text-embedding-ada-002')
    print (f"Embedding for query='{query}', embeddding_length={len(embedding)}, printing first few numbers... :\n", embedding [:10] )

    embeddings[query] = embedding

In [ ]:
import json

str = json.dumps(embeddings)

with open("embeddings_openai.json", "w") as f:
    f.write(str)

print ("saved to : 'embeddings_openai.json'")

saved to : 'embeddings_openai.json'


In [ ]:
import os
import json

cached_embeddings = {}
cached_embedding_file = 'embeddings_openai.json'

if os.path.exists(cached_embedding_file):
    with open(cached_embedding_file, "r") as f:
        str = f.read()
        cached_embeddings = json.loads(str)

print ("Loaded the following cached embeddings...")
for query in cached_embeddings.keys():
    print (f'- {query}')

Loaded the following cached embeddings...
- What is GPT-4?
- What is GPT-3.5?
- What is GPT-3.5 in terms of capabilities?
- What is GPT-4 in terms of capabilities?


Need to update AtlasClient.py to include 'text' : 1

In [ ]:
import time

# Handy function
def do_vector_search (query:str) -> None:
    query = query.lower().strip()
    print ('query: ', query)
    if query in cached_embeddings.keys():
        print ("using cached embeddings")
        embedding = cached_embeddings.get (query)
    else:
        t1a = time.perf_counter()
        embedding = openAI_client.get_embedding(query)
        print(embedding)
        t1b = time.perf_counter()
        print (f"Getting embeddings from OpenAI took {(t1b-t1a)*1000:,.0f} ms")

    t2a = time.perf_counter()
    chunks = atlas_client.vector_search(collection_name=COLLECTION_NAME, index_name=INDEX_NAME, attr_name='embedding',embedding_vector=embedding,limit=10 )
    t2b = time.perf_counter()

    print (f"Altas query returned {len (chunks)} movies in {(t2b-t2a)*1000:,.0f} ms")

    for idx, chunk in enumerate (chunks):
         print(f'{idx+1}\nid: {chunk["_id"]}\ntext: {chunk["text"]}' +
         f'\nsearch_score(meta):{chunk["search_score"]}n')

In [ ]:
query="What is GPT-4?"

do_vector_search (query=query)